In [28]:
import requests
from bs4 import BeautifulSoup
import csv
from itertools import cycle
from statistics import mean 
import random
from random import randint
import numpy as np
import time
from datetime import date

In [29]:
symbols = []
companyNames = []
companySectors = []
userAgentList = []

useragents = open("useragents.txt", "r")

for line in useragents:
    userAgentList.append(line.replace('\n', ''))   

In [30]:
def getProxies(inURL):
    
    page = requests.get(inURL)
    soup = BeautifulSoup(page.text, 'html.parser')
    terms = soup.find_all('tr')
    IPs = []

    for x in range(len(terms)):  
        
        term = str(terms[x])        
        
        if '<tr><td>' in str(terms[x]):
            pos1 = term.find('d>') + 2
            pos2 = term.find('</td>')

            pos3 = term.find('</td><td>') + 9
            pos4 = term.find('</td><td>US<')
            
            IP = term[pos1:pos2]
            port = term[pos3:pos4]
            
            if '.' in IP and len(port) < 6:
                IPs.append(IP + ":" + port)
                #print(IP + ":" + port)

    return IPs 


proxyURL = "https://www.us-proxy.org/"
pxs = getProxies(proxyURL)
proxyPool = cycle(pxs)

In [31]:
def getSymbolsCSV(fileName):
    
    global symbols
    
    print('\nLoading data from ' + fileName)

    with open(fileName) as csvfile:
    
        readCSV = csv.reader(csvfile, delimiter=',')
        
        for row in readCSV:
            
            if row[0] =='ï»¿A':
                symbols.append('A')
            else:
                symbols.append(row[0])             

In [82]:
overall = []

def getOptionsSpreads(sym):
    
    reload = True
    
    while reload == True:
        
        url = 'https://finance.yahoo.com/quote/' + sym + '/options?date=1613088000'
        #print(url)

        agent = random.choice(userAgentList)
        headers = {'User-Agent': agent}

        page = requests.get(url, headers=headers, proxies = {"http": next(proxyPool)})
        soup = BeautifulSoup(page.text, 'html.parser')

        #print(soup)

        optionChain = soup.find(id='Col1-1-OptionContracts-Proxy')

        if optionChain != None:
            
            options = optionChain.find_all('tr', class_='in-the-money')
            reload = False
            calls = []

            for x in options:
                
                x = str(x)
                
                quote = x[x.find('href="/quote/') : x.find('href="/quote/') + 100]
                
                if 'P00' in quote:
                    break
                    
                else:
                    
                    strike = x[x.find('options?strike=') + 15 : x.find('options?strike=') + 22]
                    strike = strike.replace('&amp', '').replace('&a', '')

                    askPrice = x[x.find('options?strike=') + 250 : x.find('class="data-col6 ')]
                    askPrice = askPrice[askPrice.find('>') + 1 : askPrice.find('<')]
                    
                    #print('MSFT', strike, askPrice)

                    calls.append([strike, askPrice])
            
            
            spreads = []
            
            
            for y in range(len(calls)):
                
                buyCallStrike = calls[y][0]
                buyCallAskPrice = calls[y][1]
                
                for z in range(y + 1, len(calls)):
                
                    sellCallStrike = calls[z][0]
                    sellCallAskPrice = calls[z][1]
                    
                    strikeDiff = float(sellCallStrike) - float(buyCallStrike)
                    premium = round(((-1 * float(buyCallAskPrice)) + float(sellCallAskPrice)) * 100, 2)
                    
                    maxProfit = strikeDiff * 100 + premium
                    
                    #print('(' + str(buyCallStrike) + ', ' + str(buyCallAskPrice) + ')', '(' + str(sellCallStrike) + ', ' + str(sellCallAskPrice) + ')', strikeDiff, premium, maxProfit)

            
                    spreads.append([buyCallStrike, buyCallAskPrice, sellCallStrike, sellCallAskPrice, strikeDiff, premium, maxProfit])
            
            
            
            spreads = sorted(spreads,key=lambda l:l[-1], reverse=True)
            
            for s in spreads:
                print(s)
                
    
getOptionsSpreads('MSFT')

['230', '12.80', '240', '3.70', 10.0, -910.0, 90.0]
['212.5', '30.35', '240', '3.70', 27.5, -2665.0, 85.0]
['227.5', '15.40', '240', '3.70', 12.5, -1170.0, 80.0]
['225', '18.00', '240', '3.70', 15.0, -1430.0, 70.0]
['230', '12.80', '232.5', '11.00', 2.5, -180.0, 70.0]
['212.5', '30.35', '217.5', '26.00', 5.0, -435.0, 65.0]
['212.5', '30.35', '232.5', '11.00', 20.0, -1935.0, 65.0]
['215', '28.05', '240', '3.70', 25.0, -2435.0, 65.0]
['205', '38.10', '240', '3.70', 35.0, -3440.0, 60.0]
['212.5', '30.35', '222.5', '20.95', 10.0, -940.0, 60.0]
['227.5', '15.40', '232.5', '11.00', 5.0, -440.0, 60.0]
['235', '8.10', '240', '3.70', 5.0, -440.0, 60.0]
['200', '43.15', '240', '3.70', 40.0, -3945.0, 55.0]
['220', '23.15', '240', '3.70', 20.0, -1945.0, 55.0]
['210', '33.20', '240', '3.70', 30.0, -2950.0, 50.0]
['225', '18.00', '232.5', '11.00', 7.5, -700.0, 50.0]
['180', '63.25', '240', '3.70', 60.0, -5955.0, 45.0]
['215', '28.05', '217.5', '26.00', 2.5, -205.0, 45.0]
['215', '28.05', '232.5', '1